# Install ELI5

In [0]:
pip install eli5

Python interpreter will be restarted.
Python interpreter will be restarted.


# Imports

In [0]:
import sklearn
import pandas as pd
import eli5
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from eli5 import show_weights, show_prediction ,explain_weights
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [0]:
iliwycmbd_features = spark.table('charging_stations.iliwycmbd_features')
mdpf=spark.table('charging_stations.mdpf_features').drop('lat','long')
bitamss= spark.table('charging_stations.bitamss').drop('lat','long')
targets=spark.table('charging_stations.targets')

modelTable= mdpf.join(iliwycmbd_features, ['tractcode','state','evcount','county','isev'], 'left')
modelTable1= modelTable.join(bitamss, ['tractcode','state','evcount','county','isev'], 'left').drop('tractcode', 'state', 'county', 'closest_ev_tract', 'transport_count', 'population', 'placekey', 'daily_top_brand', 'monthly_top_brand', 'county_name').toPandas()
targets=spark.table('charging_stations.targets')
targets= targets.drop('evcount','tractcode','state','county').toPandas()


In [0]:
y=targets
X= modelTable1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state = 14)


model = xgb.XGBRegressor(random_state=14)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score_original = r2_score(y_test, y_pred)
r2_score_original

Out[3]: 0.05377815038110201

In [0]:
show_weights(model)

Weight,Feature
0.0809,distance_km
0.0567,median_gas_dwell_time
0.0428,avg_gas_station_visits_on_weekday
0.0410,daily_top_brand_visits
0.0266,monthly_top_brand_visits
0.0220,land_m
0.0184,car_truck_or_van_carpool
0.0182,total_visit_counts
0.0178,car_truck_or_van_7_more_person
0.0177,car_truck_or_van_alone


In [0]:
show_prediction(model, X.iloc[700], show_feature_values=True, top=(5,5))
# This shows the top 5 and bottom 5 features. 

y
 
 

 
 (score -0.300 )

top features
 
 
 
 
 
 
 
 Contribution ? 
 
 
 Feature 
 
 Value 
 
 
 
 
 
 
 
 +0.057
 
 
 distance_km
 
 
 
 2.827
 
 
 
 
 
 
 +0.008
 
 
 count_fam_homes
 
 
 
 3622.000
 
 
 
 
 
 
 +0.006
 
 
 total_visit_counts
 
 
 
 692949.000
 
 
 
 
 
 
 +0.003
 
 
 percentage
 
 
 
 0.000
 
 
 
 
 
 
 +0.002
 
 
 subway_avg_daily_traffic
 
 
 
 Missing
 
 
 
 
 
 
 
 … 20 more positive … 
 
 
 

 
 
 
 … 25 more negative … 
 
 
 
 
 
 
 -0.014
 
 
 monthly_top_brand_visits
 
 
 
 147.000
 
 
 
 
 
 
 -0.015
 
 
 median_gas_dwell_time
 
 
 
 7.110
 
 
 
 
 
 
 -0.016
 
 
 square_miles
 
 
 
 12.530
 
 
 
 
 
 
 -0.016
 
 
 avg_gas_station_visits_on_weekday
 
 
 
 114.000
 
 
 
 
 
 
 -0.265
 
 
 <BIAS>
 
 
 
 1.000

In [0]:
show_prediction(model, X.iloc[700], show_feature_values=True,feature_re=r"^t" ) 
# this shows all feature predictions but its filtered by everything starting with "t"

y
 
 

 
 (score -0.300 )

top features
 
 
 
 
 
 
 
 Contribution ? 
 
 
 Feature 
 
 Value 
 
 
 
 
 
 
 
 +0.006
 
 
 total_visit_counts
 
 
 
 692949.000
 
 
 
 
 
 
 +0.000
 
 
 trans_NA
 
 
 
 2650.000
 
 
 
 
 
 
 +0.000
 
 
 transport_station_counts
 
 
 
 0.000
 
 
 
 
 

 
 
 
 
 -0.000
 
 
 total_tract_population
 
 
 
 5171.000
 
 
 
 
 
 
 -0.000
 
 
 total_starbucks_by_tract
 
 
 
 Missing
 
 
 
 
 
 
 -0.001
 
 
 taxi
 
 
 
 0.000

In [0]:
show_weights(model, feature_filter=lambda fet : fet.endswith("n"))
# This does the exact oppisite of the above function. gets everything that ends with "n"

Weight,Feature
0.0178,car_truck_or_van_7_more_person
0.0169,car_truck_or_van_3_person
0.0164,population_per_gas_station
0.0155,car_truck_or_van
0.0153,car_truck_or_van_carpool_2_person
0.0147,car_truck_or_van_4_person
0.0141,car_truck_or_van_5_6_person
0.0140,total_tract_population
0.0131,public_trans_train


# Housing example

In [0]:
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
#housing.info()
#Before we do anything to the data lets see how good the model does just off the basic data
y_before = housing.price
X = housing.drop(columns=['id','date','price'])
X_train, X_test, y_train, y_test = train_test_split(X, y_before, test_size=.2, random_state = 14)


model = xgb.XGBRegressor(random_state=14)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score_original = r2_score(y_test, y_pred)# best score so far 0.9981473851472684
r2_score_original


Out[8]: 0.8927099331051527

In [0]:

show_weights(model)

Weight,Feature
0.4605,grade
0.2345,waterfront
0.1117,sqft_living
0.0533,lat
0.0299,view
0.0262,long
0.0161,yr_built
0.0142,zipcode
0.0128,sqft_living15
0.0082,sqft_above


In [0]:
show_prediction(model, X.iloc[708], show_feature_values=True)

y
 
 

 
 (score 458161.229 )

top features
 
 
 
 
 
 
 
 Contribution ? 
 
 
 Feature 
 
 Value 
 
 
 
 
 
 
 
 +540546.599
 
 
 <BIAS>
 
 
 
 1.000
 
 
 
 
 
 
 +38474.841
 
 
 lat
 
 
 
 47.713
 
 
 
 
 
 
 +14588.918
 
 
 yr_built
 
 
 
 1941.000
 
 
 
 
 
 
 +6750.933
 
 
 condition
 
 
 
 4.000
 
 
 
 
 
 
 +4767.062
 
 
 sqft_lot
 
 
 
 9360.000
 
 
 
 
 
 
 +3428.585
 
 
 long
 
 
 
 -122.283
 
 
 
 
 
 
 +2019.823
 
 
 bedrooms
 
 
 
 3.000
 
 
 
 
 
 
 +1976.741
 
 
 floors
 
 
 
 1.000
 
 
 
 
 
 
 +783.795
 
 
 sqft_lot15
 
 
 
 7200.000
 
 
 
 
 
 
 +175.456
 
 
 bathrooms
 
 
 
 1.750
 
 
 
 
 

 
 
 
 
 -231.319
 
 
 waterfront
 
 
 
 0.000
 
 
 
 
 
 
 -2241.041
 
 
 yr_renovated
 
 
 
 0.000
 
 
 
 
 
 
 -5944.459
 
 
 sqft_above
 
 
 
 940.000
 
 
 
 
 
 
 -7103.751
 
 
 view
 
 
 
 0.000
 
 
 
 
 
 
 -7479.898
 
 
 zipcode
 
 
 
 98125.000
 
 
 
 
 
 
 -11595.488
 
 
 sqft_living
 
 
 
 1880.000
 
 
 
 
 
 
 -13477.596
 
 
 sqft_living15
 
 
 
 1390.000
 
 
 
 
 
 
 -19283.361
 
 
 sqft_basement
 
 
 
 940.000
 
 
 
 
 
 
 -87994.612
 
 
 grade
 
 
 
 7.000

In [0]:
explain_weights(model)

Weight,Feature
0.4605,grade
0.2345,waterfront
0.1117,sqft_living
0.0533,lat
0.0299,view
0.0262,long
0.0161,yr_built
0.0142,zipcode
0.0128,sqft_living15
0.0082,sqft_above
